In [41]:
import pandas as pd
from numpy import genfromtxt
from PIL import Image
import torch
import numpy as np
from copy import deepcopy as copy
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Checkpoint 1

In [65]:
df = pd.read_csv ('../digit-recognizer/train.csv')
print (df.columns)

Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=785)


In [66]:
print(df.iloc[0])

label       1
pixel0      0
pixel1      0
pixel2      0
pixel3      0
           ..
pixel779    0
pixel780    0
pixel781    0
pixel782    0
pixel783    0
Name: 0, Length: 785, dtype: int64


In [67]:
my_data = genfromtxt('../digit-recognizer/train.csv', delimiter=',')

In [68]:
example = my_data[2][1:].reshape((28,28)) # first column contains labels

In [69]:
img = Image.fromarray(example)
img = img.convert('L')
img

In [119]:
my_data = torch.from_numpy(my_data)

TypeError: expected np.ndarray (got Tensor)

In [120]:
trainset = torch.utils.data.DataLoader(my_data, batch_size=8, shuffle=True)

In [121]:
for data in trainset:
    print(data)
    break

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [9., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [3., 0., 0.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [9., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)


# Checkpoint 2

In [2]:
numpy_raw = genfromtxt('../digit-recognizer/train.csv', delimiter=',')

In [3]:
print(numpy_raw.shape)

(42001, 785)


In [4]:
data = numpy_raw[1:]
examples = []
labels = []
for datum in data:
    labels.append(datum[0])
    examples.append(datum[1:].reshape(28,28))

In [9]:
print(len(labels))
print(len(examples))
print(labels[0])
print(type(examples[0]))

42000
42000
1.0
<class 'numpy.ndarray'>


In [38]:
class Net(nn.Module):
    
    # defines layers
    def __init__(self):
        super().__init__()  #in    #out
        self.fc1 = nn.Linear(28*28, 64)  # fc stands for fully connected, 28*28 is flattened input value, 64 is connections between hidden layers
        self.fc2 = nn.Linear(64, 64)  # input is output of previous layer
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 10)  # output is classification values (0 - 9)
        
    # how we want data flows through network
    def forward(self, x):
        x = F.relu(self.fc1(x))  # running activation function on layer
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)  # don't want relu on last layer as want probability distribution for classification (softmax for multiclass)
        return F.log_softmax(x, dim=1)

In [103]:
X = examples[0]
X = torch.tensor(examples[0]).float()
print(X.shape)
X = X.view(1, 28*28)
print(X.shape)

net = Net()
output = net(X)
print(output) # crappy predictions, but example of functional NN

torch.Size([28, 28])
torch.Size([1, 784])
tensor([[-1.7499, -4.5126, -4.3711, -2.5392, -5.1446, -3.9986, -4.2952, -3.2694,
         -0.4580, -4.1795]], grad_fn=<LogSoftmaxBackward>)


In [104]:
train = torch.utils.data.DataLoader(data, batch_size=8, shuffle=True)

In [105]:
optimizer = optim.Adam(net.parameters(), lr=0.001)  # parameters to optimize, learning rate
epochs = 2

for epoch in range(epochs):
    net.zero_grad()
    loss = 0
    for data in train:
        for datum in data:
            x = datum[1:].float()
            x = x.view(1, 28*28)
            y = datum[0].long()
            y = y.view(1)
            output = net(x)
            loss += F.nll_loss(output, y)  #nll loss for y scalars, means squared error for a vector
    loss.backward()
    optimizer.step()  # adjusts weights
    print(loss)

tensor(54.6163, grad_fn=<AddBackward0>)
tensor(16.0126, grad_fn=<AddBackward0>)


In [106]:
correct = 0
total = 0
with torch.no_grad(): # doesn't train with gradient, useful for evaluating on testing data
    for data in train:
        for datum in data:
            x = datum[1:].float()
            x = x.view(1, 28*28)
            y = datum[0].long()
            y = y.view(1)
            output = net(x)
            if torch.argmax(output) == y:
                correct += 1
            total += 1
print('Accuracy: ', round(correct/total, 3))

Accuracy:  0.875
